In [15]:
# Creating and demonstrating the requested functions for precomputed time-series analysis.
# This code will:
# 1. Define load_data(file_path, col, date_col='Date') that loads the CSV and prepares a DataFrame.
# 2. Define precompute_basic_features(df, price_col, windows=[7,30], rsi_window=14) that computes:
#    - rolling mean, median, std for the windows
#    - daily returns and cumulative returns
#    - SMA (rolling mean) and EMA
#    - rolling correlation with Volume (if present)
#    - volatility (std of returns)
#    - RSI and MACD
# 3. Define save_pre_analysis(df, original_path) to save to a new CSV with suffix "_pre_analysis".
# 4. Demonstrate running these functions on /mnt/data/goog_stock_data.csv using 'Close' if present.
# Note: this environment has no internet access. This will run locally and produce a CSV in /mnt/data.

import pandas as pd
import numpy as np
import os
from typing import List

def load_data(file_path: str, col: str, date_col: str = "Date", parse_dates: bool = True) -> pd.DataFrame:
    """
    Load CSV into a pandas DataFrame, set the date column as a datetime index (if present),
    and ensure the requested column exists.
    
    Parameters
    ---------
    file_path : str
        Path to the CSV file.
    col : str
        Name of the price column to focus on (e.g., 'Close').
    date_col : str
        Name of the date column. Defaults to 'Date'.
    parse_dates : bool
        Whether to parse the date column as datetime.
    
    Returns
    -------
    pd.DataFrame
        Loaded DataFrame with a datetime index (if date_col exists) and containing the requested column.
    """
    df = pd.read_csv(file_path)
    df["Date"] = pd.to_datetime(df["Date"], utc=True)
    df = df.set_index("Date").sort_index()
    # parse date column and set index if exists
    if parse_dates and date_col in df.columns:
        df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
        if df[date_col].isnull().all():
            # failed parsing, keep as-is
            df = df.copy()
        else:
            df = df.set_index(date_col).sort_index()
    # ensure the requested column exists; if not, try to pick a sensible default
    if col not in df.columns:
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        fallback = numeric_cols[0] if numeric_cols else None
        raise ValueError(f"Column '{col}' not found in file. Numeric columns available: {numeric_cols}. "
                         f"Consider using one of those (e.g. '{fallback}').")
    return df

def _ema(series: pd.Series, span: int) -> pd.Series:
    return series.ewm(span=span, adjust=False).mean()

def precompute_basic_features(df: pd.DataFrame, price_col: str,
                              windows: List[int] = [7, 30],
                              rsi_window: int = 14) -> pd.DataFrame:
    """
    Compute basic statistical/time-series features described by the user:
      - rolling mean, median, std for windows
      - daily returns and cumulative returns
      - SMA (rolling mean) and EMA
      - rolling correlations with Volume if present
      - volatility (std of returns)
      - RSI and MACD
    
    Returns a new DataFrame with added columns.
    """
    df = df.copy()
    price = df[price_col].astype(float)
    
    # Daily returns
    df['daily_return'] = price.pct_change()
    # Cumulative return from start
    df['cumulative_return'] = (1 + df['daily_return']).cumprod() - 1

    # Rolling stats and MAs/EMAs
    for w in windows:
        df[f'rolling_mean_{w}'] = price.rolling(window=w, min_periods=1).mean()
        df[f'rolling_median_{w}'] = price.rolling(window=w, min_periods=1).median()
        df[f'rolling_std_{w}'] = price.rolling(window=w, min_periods=1).std(ddof=0)
        # EMA and SMA (SMA = rolling mean)
        df[f'ema_{w}'] = _ema(price, span=w)
        df[f'sma_{w}'] = df[f'rolling_mean_{w}']

    # Volatility: std of returns over windows
    for w in windows:
        df[f'volatility_{w}'] = df['daily_return'].rolling(window=w, min_periods=1).std(ddof=0)

    # Rolling correlation with volume if volume exists
    if 'Volume' in df.columns:
        for w in windows:
            df[f'rolling_corr_price_volume_{w}'] = price.rolling(window=w, min_periods=1).corr(df['Volume'])

    # RSI implementation (Wilder's smoothing)
    delta = price.diff()
    up = delta.clip(lower=0.0)
    down = -1 * delta.clip(upper=0.0)
    # Use exponential moving average of gains/losses (Wilder's)
    roll_up = up.ewm(alpha=1/rsi_window, adjust=False, min_periods=1).mean()
    roll_down = down.ewm(alpha=1/rsi_window, adjust=False, min_periods=1).mean()
    rs = roll_up / (roll_down.replace(0, np.nan))
    df['rsi'] = 100 - (100 / (1 + rs))
    df['rsi'] = df['rsi'].fillna(0)

    # MACD: EMA12 - EMA26 and signal line 9-day EMA of MACD
    ema_short = _ema(price, span=12)
    ema_long = _ema(price, span=26)
    df['macd'] = ema_short - ema_long
    df['macd_signal'] = _ema(df['macd'].fillna(0), span=9)
    df['macd_histogram'] = df['macd'] - df['macd_signal']
    
    # Momentum: difference between current price and price n periods ago (we'll include for windows)
    for w in windows:
        df[f'momentum_{w}'] = price - price.shift(w)
    
    # Clean up infinite values if any and keep numeric columns consistent
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    return df

def save_pre_analysis(df: pd.DataFrame, original_path: str) -> str:
    """
    Save dataframe to a new CSV with suffix "_pre_analysis" before the file extension.
    Returns the path to the saved CSV.
    """
    base, ext = os.path.splitext(original_path)
    new_path = f"{base}_pre_analysis{ext}"
    df.to_csv(new_path, index=True)
    return new_path

# --- Demonstration on the provided file (if present) ---
# input_path = "/mnt/data/goog_stock_data.csv"
# demo_output_path = None
# try:
#     # Try to load using 'Close' as the price column; if not present, pick a numeric column automatically.
#     try:
#         demo_df = load_data(input_path, col='Close', date_col='Date')
#         chosen_col = 'Close'
#     except ValueError as e:
#         # If 'Close' not present, inspect columns and choose first numeric column
#         tmp = pd.read_csv(input_path)
#         numeric_cols = tmp.select_dtypes(include=[np.number]).columns.tolist()
#         if not numeric_cols:
#             raise RuntimeError("No numeric columns found in the uploaded CSV; cannot compute time-series features.")
#         chosen_col = numeric_cols[-1]  # choose last numeric column as fallback
#         demo_df = load_data(input_path, col=chosen_col, date_col='Date')
    
#     analyzed = precompute_basic_features(demo_df, price_col=chosen_col, windows=[7, 30], rsi_window=14)
#     demo_output_path = save_pre_analysis(analyzed, input_path)
#     print("✅ Pre-analysis completed.")
#     print(f"Input file: {input_path}")
#     print(f"Using price column: {chosen_col}")
#     print(f"Saved output to: {demo_output_path}")
#     # show top rows
#     analyzed.head(8)
# except Exception as e:
#     print("❌ Error during demonstration:", str(e))


In [16]:
import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from langchain_core.documents import Document
import warnings
warnings.filterwarnings("ignore")

# -------------------------
# B. Trend & Seasonality
# -------------------------
def compute_trend_seasonality(df: pd.DataFrame, price_col: str, model: str = 'additive', period: int = 30) -> pd.DataFrame:
    df = df.copy()
    df = df.dropna(subset=[price_col])
    result = seasonal_decompose(df[price_col], model=model, period=period, extrapolate_trend='freq')
    df['trend'] = result.trend
    df['seasonal'] = result.seasonal
    df['residual'] = result.resid
    return df

# -------------------------
# C. Anomaly Detection
# -------------------------
def detect_anomalies(df: pd.DataFrame, price_col: str, z_thresh: float = 3.0) -> pd.DataFrame:
    df = df.copy()
    # Z-score method
    df['zscore'] = (df[price_col] - df[price_col].mean()) / df[price_col].std(ddof=0)
    df['zscore_anomaly'] = df['zscore'].abs() > z_thresh

    # Isolation Forest
    scaler = StandardScaler()
    scaled_vals = scaler.fit_transform(df[[price_col]].fillna(0))
    iso = IsolationForest(contamination=0.01, random_state=42)
    df['isolation_anomaly'] = iso.fit_predict(scaled_vals) == -1
    return df

# -------------------------
# D. Forecasting
# -------------------------
def forecast_arima(df: pd.DataFrame, price_col: str, steps: int = 7):
    df = df.dropna(subset=[price_col])
    model = ARIMA(df[price_col], order=(5,1,0))
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=steps)
    forecast_df = pd.DataFrame({
        'forecast_date': pd.date_range(df.index[-1] + pd.Timedelta(days=1), periods=steps),
        'forecast_arima': forecast.values
    })
    return forecast_df

def forecast_lstm(df: pd.DataFrame, price_col: str, steps: int = 7, epochs: int = 5):
    from sklearn.preprocessing import MinMaxScaler
    from numpy import array

    prices = df[price_col].dropna().values.reshape(-1, 1)
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(prices)

    X, y = [], []
    for i in range(len(scaled)-1):
        X.append(scaled[i])
        y.append(scaled[i+1])
    X, y = np.array(X), np.array(y)
    X = X.reshape((X.shape[0], 1, X.shape[1]))

    model = Sequential([
        LSTM(50, activation='relu', input_shape=(1,1)),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X, y, epochs=epochs, verbose=0)

    # forecast
    last_val = scaled[-1].reshape((1,1,1))
    preds = []
    for _ in range(steps):
        next_pred = model.predict(last_val, verbose=0)
        preds.append(next_pred[0,0])
        last_val = next_pred.reshape((1,1,1))
    preds = scaler.inverse_transform(np.array(preds).reshape(-1,1)).flatten()
    forecast_df = pd.DataFrame({
        'forecast_date': pd.date_range(df.index[-1] + pd.Timedelta(days=1), periods=steps),
        'forecast_lstm': preds
    })
    return forecast_df

# -------------------------
# E. Summary for Embedding
# -------------------------
def create_langchain_summaries(df: pd.DataFrame, price_col: str, freq: str = 'M') -> list:
    """
    Create multiple LangChain Documents summarizing trends/anomalies per time chunk (e.g., month).
    """
    docs = []
    
    # ✅ Ensure the DataFrame has a DatetimeIndex
    if not isinstance(df.index, pd.DatetimeIndex):
        # Try to auto-detect a date column
        date_cols = [c for c in df.columns if 'date' in c.lower()]
        if date_cols:
            df[date_cols[0]] = pd.to_datetime(df[date_cols[0]], errors='coerce')
            df = df.set_index(date_cols[0])
        else:
            raise ValueError("No DatetimeIndex or 'Date' column found for resampling.")
    
    grouped = df.resample(freq)
    for period, group in grouped:
        if len(group) < 5:
            continue
        mean_price = group[price_col].mean()
        vol = group['daily_return'].std() if 'daily_return' in group else group[price_col].pct_change().std()
        anomalies = group['zscore_anomaly'].sum() if 'zscore_anomaly' in group else 0
        trend_desc = "increasing" if group['trend'].iloc[-1] > group['trend'].iloc[0] else "decreasing"

        text = (
            f"Between {group.index.min().date()} and {group.index.max().date()}, "
            f"the average {price_col} was {mean_price:.2f}. "
            f"The overall trend was {trend_desc}. "
            f"Volatility measured by return std was {vol:.4f}. "
            f"{anomalies} anomalies were detected this period."
        )

        meta = {
            "period": str(period.date()),
            "mean_price": mean_price,
            "volatility": vol,
            "num_anomalies": int(anomalies)
        }
        docs.append(Document(page_content=text, metadata=meta))
    return docs


In [17]:
col = "Close"
df = load_data("../data/goog_stock_data.csv", col="Close")
print(df.index)
df = precompute_basic_features(df, "Close")

# B & C
df = compute_trend_seasonality(df, "Close")
df = detect_anomalies(df, "Close")

# D
arima_forecast = forecast_arima(df, "Close")
lstm_forecast = forecast_lstm(df, "Close", epochs=5)

# Merge forecasts if needed
df_forecasts = pd.merge(arima_forecast, lstm_forecast, on="forecast_date", how="outer")

# E
docs = create_langchain_summaries(df, "Close")
print(docs[0].page_content)
print(docs[0].metadata)


DatetimeIndex(['2020-10-26 04:00:00+00:00', '2020-10-27 04:00:00+00:00',
               '2020-10-28 04:00:00+00:00', '2020-10-29 04:00:00+00:00',
               '2020-10-30 04:00:00+00:00', '2020-11-02 05:00:00+00:00',
               '2020-11-03 05:00:00+00:00', '2020-11-04 05:00:00+00:00',
               '2020-11-05 05:00:00+00:00', '2020-11-06 05:00:00+00:00',
               ...
               '2025-10-13 04:00:00+00:00', '2025-10-14 04:00:00+00:00',
               '2025-10-15 04:00:00+00:00', '2025-10-16 04:00:00+00:00',
               '2025-10-17 04:00:00+00:00', '2025-10-20 04:00:00+00:00',
               '2025-10-21 04:00:00+00:00', '2025-10-22 04:00:00+00:00',
               '2025-10-23 04:00:00+00:00', '2025-10-24 04:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='Date', length=1256, freq=None)
Between 2020-10-26 and 2020-10-30, the average Close was 78.46. The overall trend was increasing. Volatility measured by return std was 0.0418. 0 anomalies were detected 

In [22]:
# ==============================================
# vector_pipeline.py
# ==============================================

from typing import List, Union
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.utils import embedding_functions

# -------------------------------------------------
# 1️⃣ Embedding Engine
# -------------------------------------------------
class EmbeddingEngine:
    """
    A wrapper around SentenceTransformer for encoding LangChain Documents.
    """

    def __init__(self, model_name: str = "all-MiniLM-L6-v2", device: str = "cpu"):
        self.model_name = model_name
        self.device = device
        self.model = SentenceTransformer(model_name, device=device)
        print(f"✅ Loaded embedding model: {model_name} on {device}")

    def embed_documents(self, docs: List[Document]) -> np.ndarray:
        """
        Encodes a list of LangChain Document objects into dense vectors.
        """
        texts = [doc.page_content for doc in docs]
        embeddings = self.model.encode(texts, batch_size=32, show_progress_bar=True, convert_to_numpy=True)
        return embeddings

    def embed_query(self, query: str) -> np.ndarray:
        """
        Encodes a single query string into a dense vector.
        """
        return self.model.encode([query], convert_to_numpy=True)[0]


# -------------------------------------------------
# 2️⃣ Vector Store (ChromaDB)
# -------------------------------------------------
class VectorStore:
    """
    A persistent vector store using ChromaDB.
    """

    def __init__(self, persist_dir: str = "./chroma_store", collection_name: str = "timeseries_docs", model_name: str = "all-MiniLM-L6-v2"):
        self.client = chromadb.PersistentClient(path=persist_dir)
        self.model_name = model_name
        self.embedding_fn = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=model_name)
        self.collection = self.client.get_or_create_collection(
            name=collection_name,
            embedding_function=self.embedding_fn
        )
        print(f"✅ Connected to ChromaDB collection: {collection_name}")

    def add_documents(self, docs: List[Document], embeddings: np.ndarray):
        """
        Adds documents + embeddings to Chroma collection.
        """
        ids = [f"doc_{i}" for i in range(len(docs))]
        metadatas = [doc.metadata for doc in docs]
        texts = [doc.page_content for doc in docs]
        self.collection.add(documents=texts, embeddings=embeddings.tolist(), metadatas=metadatas, ids=ids)
        print(f"📚 Added {len(docs)} documents to ChromaDB")

    def query(self, query_embedding: np.ndarray, top_k: int = 3):
        """
        Retrieves top-k similar documents given a query embedding.
        """
        results = self.collection.query(query_embeddings=[query_embedding.tolist()], n_results=top_k)
        hits = [
            {"text": results["documents"][0][i], "score": results["distances"][0][i], "metadata": results["metadatas"][0][i]}
            for i in range(len(results["documents"][0]))
        ]
        return hits

    def clear(self):
        """Clears all documents in the ChromaDB collection safely."""
        all_ids = self.collection.get()["ids"]
        if all_ids:
            self.collection.delete(ids=all_ids)
            print(f"🧹 Cleared {len(all_ids)} documents from ChromaDB collection.")
        else:
            print("🧹 Collection already empty.")


# -------------------------------------------------
# 3️⃣ Example Pipeline Usage
# -------------------------------------------------
# if __name__ == "__main__":
#     # Assuming 'docs' is your list of LangChain Documents
#     from my_pipeline import docs   # Replace this with your own import

# Step 1: Embed
embedder = EmbeddingEngine(model_name="all-MiniLM-L6-v2")
doc_embeddings = embedder.embed_documents(docs)

# Step 2: Store in Chroma
store = VectorStore(persist_dir="./chroma_store", collection_name="goog_timeseries")
store.clear()
store.add_documents(docs, doc_embeddings)

    # Step 3: Query
user_query = "Explain the volatility trend for March 2024"
query_vec = embedder.embed_query(user_query)
results = store.query(query_vec, top_k=3)

for r in results:
    print("\n🧩 Match:")
    print("Text:", r["text"])
    print("Score:", round(r["score"], 4))
    print("Metadata:", r["metadata"])


✅ Loaded embedding model: all-MiniLM-L6-v2 on cpu


Batches: 100%|██████████| 2/2 [00:00<00:00,  4.78it/s]


✅ Connected to ChromaDB collection: goog_timeseries
🧹 Collection already empty.
📚 Added 61 documents to ChromaDB

🧩 Match:
Text: Between 2022-06-01 and 2022-06-30, the average Close was 112.06. The overall trend was increasing. Volatility measured by return std was 0.0261. 0 anomalies were detected this period.
Score: 0.3013
Metadata: {'mean_price': 112.05827476864768, 'num_anomalies': 0, 'volatility': 0.026124593312125684, 'period': '2022-06-30'}

🧩 Match:
Text: Between 2025-01-02 and 2025-01-31, the average Close was 196.20. The overall trend was decreasing. Volatility measured by return std was 0.0167. 0 anomalies were detected this period.
Score: 0.3077
Metadata: {'period': '2025-01-31', 'volatility': 0.016701299623074674, 'mean_price': 196.19761352539064, 'num_anomalies': 0}

🧩 Match:
Text: Between 2022-12-01 and 2022-12-30, the average Close was 92.10. The overall trend was decreasing. Volatility measured by return std was 0.0173. 0 anomalies were detected this period.
Score: 0.3

In [12]:
import pandas as pd

df = pd.read_csv("../data/goog_stock_data.csv")
df["Date"] = pd.to_datetime(df["Date"], utc=True)
df = df.set_index("Date").sort_index()
print(df.head())
print(df.columns)
print(df.dtypes)

                                Open       High        Low      Close  \
Date                                                                    
2020-10-26 04:00:00+00:00  80.698262  81.355265  78.289241  78.982002   
2020-10-27 04:00:00+00:00  79.241239  79.796187  78.601118  79.667816   
2020-10-28 04:00:00+00:00  77.456942  77.536893  75.216279  75.315598   
2020-10-29 04:00:00+00:00  75.600633  79.143890  75.594677  77.829384   
2020-10-30 04:00:00+00:00  83.037261  83.776700  79.677751  80.499626   

                             Volume  Dividends  Stock Splits  
Date                                                          
2020-10-26 04:00:00+00:00  37066000        0.0           0.0  
2020-10-27 04:00:00+00:00  24580000        0.0           0.0  
2020-10-28 04:00:00+00:00  36680000        0.0           0.0  
2020-10-29 04:00:00+00:00  40062000        0.0           0.0  
2020-10-30 04:00:00+00:00  86582000        0.0           0.0  
Index(['Open', 'High', 'Low', 'Close', 'Volume'

In [13]:
print(df.index)

DatetimeIndex(['2020-10-26 04:00:00+00:00', '2020-10-27 04:00:00+00:00',
               '2020-10-28 04:00:00+00:00', '2020-10-29 04:00:00+00:00',
               '2020-10-30 04:00:00+00:00', '2020-11-02 05:00:00+00:00',
               '2020-11-03 05:00:00+00:00', '2020-11-04 05:00:00+00:00',
               '2020-11-05 05:00:00+00:00', '2020-11-06 05:00:00+00:00',
               ...
               '2025-10-13 04:00:00+00:00', '2025-10-14 04:00:00+00:00',
               '2025-10-15 04:00:00+00:00', '2025-10-16 04:00:00+00:00',
               '2025-10-17 04:00:00+00:00', '2025-10-20 04:00:00+00:00',
               '2025-10-21 04:00:00+00:00', '2025-10-22 04:00:00+00:00',
               '2025-10-23 04:00:00+00:00', '2025-10-24 04:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='Date', length=1256, freq=None)
